# Automobile Sales Statistics Dashboard

### Objective

The objective of this project is to build an app consisted in two dashboards about the automobile statistics report: one for the recession perido, and another for the global report. 

Both options considers four illustrations. The former includes a line chart for the Automobile sales fluctuate over Recession Period, a bar plot illusttrating the average number of vehicles sold by vehicle type, a Pie chart for total expenditure share by vehicle type, and a bar chart againg for the effect of unemployment rate on vehicle type and sales; whereas the latter displays two line plots (one for Yearly Automobile sales, and another for the total automobile sales per month), a bar plot to illustrate the average number of vehicles sold during the given year, and a pie chart representing the total advertisement expenditure.

### Building the app

In [71]:
# Importing Libraries

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [72]:
# Loading the flat file

data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

In [73]:
# Initializing the Dash app
app = dash.Dash()

In [74]:
# Setting the title of the dashboard
app.title = 'Automobile Statistics Dashboard'

In [75]:
# Creating the dropdown menu options
dropdown_options = [
    {'label': 'Yearly statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

In [76]:
# List of years 
year_list = list(data['Year'].unique())

In [77]:
# Creating the layout of the app
app.layout = html.Div([
    # Adding a title to the dashboard
    html.H1('Automobile Sales Statistics Dashboard', style={'textAlign': 'center', 'font-size': 30}),
    html.Div([# Adding two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics', 
            options=dropdown_options,
            placeholder='Select a report type',
            style={'textAlign': 'center'})
        
    ]),
    html.Div(dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in year_list],
        value=year_list[0],
        placeholder='Select a year'
    )),
    html.Div([# Adding a division for output display
        html.Div(id='output-container', className='chart-grid', style={'width': '100%'}),
    ])
])

In [78]:
# Creating Callbacks
# Defining the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property= 'disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistics):
    return selected_statistics != 'Yearly Statistics'

In [79]:
# Callback for plotting
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
     Input(component_id='dropdown-statistics', component_property='value')]
)
def update_output_container(input_year, selected_statistics):
    if selected_statistics == 'Recession Period Statistics':
        # Filtering the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # Plot 1: Automobile sales fluctuate over Recession Period (year wise)
        R_chart1 = dcc.Graph(
            figure=px.line(recession_data, x='Year', y='Automobile_Sales',
                           title="Average Automobile Sales fluctuation over Recession Period")
        )

        # Plot 2: Calculate the average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales',
                          title="Average Automobile Sales by Vehicle Type over Recession Period")
        )

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                          title="Total Expenditure by Vehicle Type")
        )

        # Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
        unemployment_rec = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].sum().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemployment_rec, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                          title="Effect of Unemployment Rate on Vehicle Type and Sales")
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)])
        ]

    elif selected_statistics == 'Yearly Statistics':
        yearly_data = data[data['Year'] == input_year]

        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, x='Year', y='Automobile_Sales',
                           title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year))
        )

        # Plot 2: Total Monthly Automobile sales using line chart.
        mas= data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas, x='Month', y='Automobile_Sales',
                           title="Monthly Average Automobile Sales fluctuation")
        )

        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                          title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year))
        )

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                          title="Total Expenditure by Vehicle Type")
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)])
        ]

    else:
        return None

### Launching the app

In [80]:
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

   
                  ________  ________   _______   ______ 
                 /_  __/ / / / ____/  / ____/ | / / __ \
                  / / / /_/ / __/    / __/ /  |/ / / / /
                 / / / __  / /___   / /___/ /|  / /_/ / 
                /_/ /_/ /_/_____/  /_____/_/ |_/_____/  